In [ ]:
# prepare your data from pdb and mol2 file
# this is a demo
from biopandas.pdb import PandasPdb
import numpy as np
from scipy.spatial import distance
import os

In [ ]:
amino3to1dict = {'ASH': 'A',
                 'ALA': 'A',
                 'CYX': 'C',
                 'CYS': 'C',
                 'ASP': 'D',
                 'GLU': 'E',
                 'PHE': 'F',
                 'GLY': 'G',
                 'HIS': 'H',
                 'HID': 'H',
                 'HIE': 'H',
                 'HIP': 'H',
                 'ILE': 'I',
                 'LYS': 'K',
                 'LEU': 'L',
                 'MET': 'M',
                 'MSE': 'M',
                 'ASN': 'N',
                 'PYL': 'O',
                 'HYP': 'P',
                 'PRO': 'P',
                 'GLN': 'Q',
                 'ARG': 'R',
                 'SER': 'S',
                 'THR': 'T',
                 'SEL': 'U',
                 'VAL': 'V',
                 'TRP': 'W',
                 'TYR': 'Y',
                 'MEN': 'X',
                 'LCX': 'X'}  

In [ ]:
with open('./Raw_data/SM/joined.txt', 'w') as out:
    for pdb in os.listdir('./data/'):
        if pdb.split('.')[-1] == 'pdb':
            ppdb = PandasPdb().read_pdb('./data/' + pdb).df
            atom = ppdb['ATOM']
            atom = atom[atom['atom_name'] == 'CA']
            seq = ''.join(list(map(lambda x: amino3to1dict[x] if x in amino3to1dict.keys() else 'x', list(atom['residue_name']))))
            atom_coord = atom[['x_coord', 'y_coord', 'z_coord']].to_numpy()

            ligand = ppdb['HETATM']
            ligand_coord = ligand[['x_coord', 'y_coord', 'z_coord']].to_numpy()

            dist_mat = distance.cdist(atom_coord, ligand_coord)
            dist_min = dist_mat.min(-1)
            bind_site = ''.join(['0' if d > 5.5 else '1' for d in dist_min])  # here 5.5 please refer to GraphBind paper.

            out.write('>' + pdb + '\n')
            out.write(seq + '\n')
            out.write(bind_site + '\n')